# Q-5: 線型計画法 (4)

## 主婦問題
https://axross-recipe.com/recipes/97

問題）food.csv にリストアップされている７７種類の食料品から何をどのくらい購入すれば、nutrient.csvに記載された１日に健康維持に必要な栄養素を最安で充足することができるか？

pulp と CVXPYを用いて求めよ。

## pulp を用いた解法

このレシピでは、数理最適化を用いて1945年にG. Stiglerによって提案された「主婦問題」を解いてみます。
「主婦問題」は、スーパー・マーケットで売られている７７種の食料品の中から、何をどのくらいの量購入すれば、健康維持に必要な栄養素を最安で充足することができるか？という問題になります

主婦問題を提唱したG. Stigerですが、実はノーベル経済学賞を受賞しています。なので、彼の研究は経済政策に紐づくことが多く、主婦問題についても、食糧支給政策を決めるためのものなのです。 それでは、当時、どのような食材を組み合わせたレシピが最適だったのか、線形計画法を用いて算出していきましょう。なお、必要な栄養下限値や、食糧の価格・栄養素は当時の米国の数値になること、ご留意ください。

In [29]:
# Google driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
# カレントディレクトリーを移動 --> 自分の Google driveでコードとデータを置いているフォルダを指定する
%cd /content/drive/MyDrive/100本ノックチャレンジ/05_Optimization_100_knocks/02_Answers
%ls -lah

/content/drive/MyDrive/100本ノックチャレンジ/05_Optimization_100_knocks/02_Answers
total 7.8M
drwx------ 2 root root 4.0K Feb  3 06:32 data/
drwx------ 2 root root 4.0K Feb  3 07:52 out/
-rw------- 1 root root  51K Feb  2 08:25 Q-1_answer_colab.ipynb
-rw------- 1 root root  52K Feb  2 08:26 Q-1_answer.ipynb
-rw------- 1 root root  45K Feb  2 10:31 Q-2_answer_colab.ipynb
-rw------- 1 root root  43K Feb  2 08:32 Q-2_answer.ipynb
-rw------- 1 root root  12K Feb  2 10:33 Q-3_answer_colab.ipynb
-rw------- 1 root root  12K Feb  2 10:31 Q-3_answer.ipynb
-rw------- 1 root root  53K Feb  3 04:29 Q-4_answer_colab.ipynb
-rw------- 1 root root  54K Feb  3 04:29 Q-4_answer.ipynb
-rw------- 1 root root  78K Feb  3 10:58 Q-5_answer_colab.ipynb
-rw------- 1 root root  71K Feb  3 06:32 Q-5_answer.ipynb
-rw------- 1 root root 3.8M Feb  3 09:55 Q-6_answer_colab.ipynb
-rw------- 1 root root 3.7M Feb  3 09:55 Q-6_answer.ipynb


In [31]:
!pip install pulp

In [32]:
from pulp import *
import numpy as np
import pandas as pd

In [33]:
df_foods = pd.read_csv("./data/food.csv")
df_foods

,food,Calories,Protein,Calcium,Iron,Vitamin A,Thiamine (Vitamin B1),Riboflavin (Vitamin B2),Niacin,Ascorbic Acid (Vitamin C)
0,Wheat Flour (Enriched),44.7,1411,2.0,365,0.0,55.4,33.3,441,0
1,Macaroni,11.6,418,0.7,54,0.0,3.2,1.9,68,0
2,Wheat Cereal (Enriched),11.8,377,14.4,175,0.0,14.4,8.8,114,0
3,Corn Flakes,11.4,252,0.1,56,0.0,13.5,2.3,68,0
4,Corn Meal,36.0,897,1.7,99,30.9,17.4,7.9,106,0
...,...,...,...,...,...,...,...,...,...,...
72,Chocolate,8.0,77,1.3,39,0.0,0.9,3.4,14,0
73,Sugar,34.9,0,0.0,0,0.0,0.0,0.0,0,0
74,Corn Syrup,14.7,0,0.5,74,0.0,0.0,0.0,5,0
75,Molasses,9.0,0,10.3,244,0.0,1.9,7.5,146,0


In [34]:
df_nutrient = pd.read_csv("./data/nutrient.csv")
df_nutrient

,Nutrient,Intake,Unit
0,Calories,3.0,kilocalories
1,Protein,70.0,grams
2,Calcium,0.8,grams
3,Iron,12.0,milligrams
4,Vitamin A,5.0,1000IU
5,Thiamine (Vitamin B1),1.8,milligrams
6,Riboflavin (Vitamin B2),2.7,milligrams
7,Niacin,18.0,milligrams
8,Ascorbic Acid (Vitamin C),75.0,milligrams


さあ、準備はできましたので、早速定式化していきましょう。以下では、問題の宣言から変数の定義まで行っています。詳細の説明はコメントをご覧ください。

In [35]:
# food_amount、食料xに何ドル使うか？という変数のリストです。これから線形最適化を使い、これら変数の最適な値を算出することになります。
# food_used[0]はWheat Flour, food_used[1]はMacaroni...という形で対応します。
# また、必要ではないですが、lowBound=0を入れて、これら変数は必ず0以上であるという条件を追加しています。
food_amount = [LpVariable('food_{0}'.format(i), lowBound=0) for i in range(len(df_foods))]
print(food_amount)

[food_0, food_1, food_2, food_3, food_4, food_5, food_6, food_7, food_8, food_9, food_10, food_11, food_12, food_13, food_14, food_15, food_16, food_17, food_18, food_19, food_20, food_21, food_22, food_23, food_24, food_25, food_26, food_27, food_28, food_29, food_30, food_31, food_32, food_33, food_34, food_35, food_36, food_37, food_38, food_39, food_40, food_41, food_42, food_43, food_44, food_45, food_46, food_47, food_48, food_49, food_50, food_51, food_52, food_53, food_54, food_55, food_56, food_57, food_58, food_59, food_60, food_61, food_62, food_63, food_64, food_65, food_66, food_67, food_68, food_69, food_70, food_71, food_72, food_73, food_74, food_75, food_76]


さて、目的関数を設定したら、次は制約を追加していきます。それぞれの栄養素の下限値を、購入した食料の栄養合計が上回るように設定していきます。

In [36]:
# 最小化問題なのでLpMinimizeを使います
problem = LpProblem('Stiger Diet', LpMinimize)

# 目的関数を設定しています。lpSumは1つのリストの総和を求めるための関数です。（numpyのsumを使っても問題ないです。）
# food_amountはその食料に何ドル使うか？の変数なので、単純に和をとれば食料費の総額になります。
problem += lpSum(food_amount)

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [37]:
print(problem)

Stiger_Diet:
MINIMIZE
1*food_0 + 1*food_1 + 1*food_10 + 1*food_11 + 1*food_12 + 1*food_13 + 1*food_14 + 1*food_15 + 1*food_16 + 1*food_17 + 1*food_18 + 1*food_19 + 1*food_2 + 1*food_20 + 1*food_21 + 1*food_22 + 1*food_23 + 1*food_24 + 1*food_25 + 1*food_26 + 1*food_27 + 1*food_28 + 1*food_29 + 1*food_3 + 1*food_30 + 1*food_31 + 1*food_32 + 1*food_33 + 1*food_34 + 1*food_35 + 1*food_36 + 1*food_37 + 1*food_38 + 1*food_39 + 1*food_4 + 1*food_40 + 1*food_41 + 1*food_42 + 1*food_43 + 1*food_44 + 1*food_45 + 1*food_46 + 1*food_47 + 1*food_48 + 1*food_49 + 1*food_5 + 1*food_50 + 1*food_51 + 1*food_52 + 1*food_53 + 1*food_54 + 1*food_55 + 1*food_56 + 1*food_57 + 1*food_58 + 1*food_59 + 1*food_6 + 1*food_60 + 1*food_61 + 1*food_62 + 1*food_63 + 1*food_64 + 1*food_65 + 1*food_66 + 1*food_67 + 1*food_68 + 1*food_69 + 1*food_7 + 1*food_70 + 1*food_71 + 1*food_72 + 1*food_73 + 1*food_74 + 1*food_75 + 1*food_76 + 1*food_8 + 1*food_9 + 0
VARIABLES
food_0 Continuous
food_1 Continuous
food_10 Continuo

In [38]:
# 制約を設定しています。lpDotは2つのリストの内積を求める関数になっています。これで、購入総額が目的関数に設定されたことになります。
for k,v in df_nutrient.iterrows():
    problem += lpDot(food_amount, df_foods[v.Nutrient]) >= v.Intake

In [39]:
print(problem)

Stiger_Diet:
MINIMIZE
1*food_0 + 1*food_1 + 1*food_10 + 1*food_11 + 1*food_12 + 1*food_13 + 1*food_14 + 1*food_15 + 1*food_16 + 1*food_17 + 1*food_18 + 1*food_19 + 1*food_2 + 1*food_20 + 1*food_21 + 1*food_22 + 1*food_23 + 1*food_24 + 1*food_25 + 1*food_26 + 1*food_27 + 1*food_28 + 1*food_29 + 1*food_3 + 1*food_30 + 1*food_31 + 1*food_32 + 1*food_33 + 1*food_34 + 1*food_35 + 1*food_36 + 1*food_37 + 1*food_38 + 1*food_39 + 1*food_4 + 1*food_40 + 1*food_41 + 1*food_42 + 1*food_43 + 1*food_44 + 1*food_45 + 1*food_46 + 1*food_47 + 1*food_48 + 1*food_49 + 1*food_5 + 1*food_50 + 1*food_51 + 1*food_52 + 1*food_53 + 1*food_54 + 1*food_55 + 1*food_56 + 1*food_57 + 1*food_58 + 1*food_59 + 1*food_6 + 1*food_60 + 1*food_61 + 1*food_62 + 1*food_63 + 1*food_64 + 1*food_65 + 1*food_66 + 1*food_67 + 1*food_68 + 1*food_69 + 1*food_7 + 1*food_70 + 1*food_71 + 1*food_72 + 1*food_73 + 1*food_74 + 1*food_75 + 1*food_76 + 1*food_8 + 1*food_9 + 0
SUBJECT TO
_C1: 44.7 food_0 + 11.6 food_1 + 12.4 food_10 + 8 f

内積の意味が掴みづらい...という方は、以下の例を参考にしていただくと分かりやすいかもしれません。

In [40]:
#下記のコードは、10*1 + 20*5 + 30*10 = 410を返す
ans = lpDot([10,20,30], [1,5,10])
print(ans)

410


それでは、目的関数と制約が用意できましたので、最適化を走らせて見ましょう。最後に、0ドル以上購入した食料だけ表示するようにしています。

In [41]:
# 解く
status = problem.solve()
print(LpStatus[status])

Optimal


In [42]:
food_list=[]
for i in range(len(food_amount)):
    food_list.append(food_amount[i].value())
np.array(food_list)

array([0.02951906, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00189256,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01121444, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.00500766, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.06102856, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [43]:
# 年間コストを算出するため、最適値に365を乗じていっています。
annual_cost = 0
for i, fa in enumerate(food_amount):
    if fa.value() > 0:
        print("{0} : ${1}".format(df_foods.food[i], fa.value()))
        annual_cost += 365 * fa.value()
        
print("上記がコスト最小となる最適な食材の組み合わせで、年間{0}ドルで済む".format(annual_cost))

Wheat Flour (Enriched) : $0.029519062
Liver (Beef) : $0.0018925573
Cabbage : $0.011214435
Spinach : $0.0050076605
Navy Beans, Dried : $0.061028564
上記がコスト最小となる最適な食材の組み合わせで、年間39.661731762ドルで済む


はい、最適解は年間コスト$39.66ということになりました。物価が違うので参考にはなりませんが、小麦粉、牛レバー、キャベツ、ほうれん草、白インゲン豆...の5つの食材を使った料理が最もコスパの良い料理ということになりますね。

実は、Google社のシェフがこの食材を元に調理したことがあるとのことで、社員曰く「美味しかった」とのことです。...、興味ある方は是非お試しください！

https://ai.googleblog.com/2014/09/sudoku-linear-optimization-and-ten-cent.html

## CVXPY を用いた解法

In [44]:
import cvxpy as cp

In [45]:
import pandas as pd
df_foods = pd.read_csv("./data/food.csv")
df_foods

,food,Calories,Protein,Calcium,Iron,Vitamin A,Thiamine (Vitamin B1),Riboflavin (Vitamin B2),Niacin,Ascorbic Acid (Vitamin C)
0,Wheat Flour (Enriched),44.7,1411,2.0,365,0.0,55.4,33.3,441,0
1,Macaroni,11.6,418,0.7,54,0.0,3.2,1.9,68,0
2,Wheat Cereal (Enriched),11.8,377,14.4,175,0.0,14.4,8.8,114,0
3,Corn Flakes,11.4,252,0.1,56,0.0,13.5,2.3,68,0
4,Corn Meal,36.0,897,1.7,99,30.9,17.4,7.9,106,0
...,...,...,...,...,...,...,...,...,...,...
72,Chocolate,8.0,77,1.3,39,0.0,0.9,3.4,14,0
73,Sugar,34.9,0,0.0,0,0.0,0.0,0.0,0,0
74,Corn Syrup,14.7,0,0.5,74,0.0,0.0,0.0,5,0
75,Molasses,9.0,0,10.3,244,0.0,1.9,7.5,146,0


In [46]:
df_nutrient = pd.read_csv("./data/nutrient.csv")
df_nutrient

,Nutrient,Intake,Unit
0,Calories,3.0,kilocalories
1,Protein,70.0,grams
2,Calcium,0.8,grams
3,Iron,12.0,milligrams
4,Vitamin A,5.0,1000IU
5,Thiamine (Vitamin B1),1.8,milligrams
6,Riboflavin (Vitamin B2),2.7,milligrams
7,Niacin,18.0,milligrams
8,Ascorbic Acid (Vitamin C),75.0,milligrams


In [47]:
# 変数の定義
food_amount = cp.Variable(len(df_foods))
food_amount

Variable((77,))

In [48]:
# 制約条件
constraints = [food_amount @ df_foods.iloc[:,i+1] >= df_nutrient.Intake[i] for i in range(len(df_nutrient))]
constraints += [food_amount[i] >= 0 for i in range(len(df_foods))]

In [49]:
constraints

[Inequality(Constant(CONSTANT, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, ZER

In [50]:
# 目的関数の定義
objective = cp.Minimize(cp.sum(food_amount))

In [51]:
# 問題の定義
prob = cp.Problem(objective, constraints)
print(prob)

minimize Sum(var1861, None, False)
subject to 3.0 <= var1861 * [44.7 11.6 11.8 11.4 36.  28.6 21.2 25.3 15.  12.2 12.4  8.  12.5  6.1
  8.4 10.8 20.6  2.9  7.4  3.5 15.7  8.6 20.1 41.7  2.9  2.2  3.4  3.6
  8.5  2.2  3.1  3.3  3.5  4.4 10.4  6.7 18.8  1.8  1.7  5.8  5.8  4.9
  1.   2.2  2.4  2.6  2.7  0.9  0.4  5.8 14.3  1.1  9.6  3.7  3.   2.4
  0.4  1.   7.5  5.2  2.3  1.3  1.6  8.5 12.8 13.5 20.  17.4 26.9  0.
  0.   8.7  8.  34.9 14.7  9.   6.4]
           70.0 <= var1861 * [1411.  418.  377.  252.  897.  680.  460.  907.  488.  484.  439.  130.
  288.  310.  422.    9.   17.  238.  448.   49.  661.   18.    0.    0.
  166.  214.  213.  309.  404.  333.  245.  140.  196.  249.  152.  212.
  164.  184.  156.  705.   27.   60.   21.   40.  138.  125.   73.   51.
   27.  166.  336.  106.  138.   20.    8.   16.   33.   54.  364.  136.
  136.   63.   71.   87.   99.  104. 1367. 1055. 1691.    0.    0.  237.
   77.    0.    0.    0.   11.]
           0.8 <= var1861 * [ 2.   0.7 14.4  0.

In [52]:
%%time
# 解く
prob.solve()  # Returns the optimal value.
print("status:", prob.status)

status: optimal
CPU times: user 105 ms, sys: 1.41 ms, total: 106 ms
Wall time: 106 ms


In [53]:
food_amount.value

array([ 2.95190617e-02,  8.92617076e-13,  4.74982920e-12,  6.02689897e-13,
        2.92958589e-12,  1.64531543e-12,  1.33629273e-12,  2.76235945e-12,
        1.48484636e-12,  1.29457260e-12,  9.97141457e-13,  1.35173940e-13,
        6.73741805e-13,  3.53036171e-12,  2.53421625e-11, -7.43432917e-13,
        9.93027426e-14,  5.67560033e-13,  6.72662026e-12, -9.00504601e-13,
        1.45891804e-12, -1.63626504e-12, -6.40003975e-13,  1.52329506e-12,
        1.66539599e-13,  3.08701282e-13,  3.22697414e-13,  5.85838457e-13,
        7.20532196e-13,  1.89255728e-03,  4.43444908e-13,  4.55634966e-14,
        2.93117735e-13,  4.84627384e-13,  2.49289687e-13,  4.25574785e-13,
        5.41571448e-13,  2.01260642e-13,  8.76978841e-14,  1.57814994e-12,
        9.37343227e-13,  8.82258259e-13,  1.07244569e-12,  1.84638441e-12,
        1.65600101e-12,  1.12144352e-02,  1.40476399e-12,  5.92823803e-13,
        8.42864349e-13,  2.05250714e-12,  4.00100634e-12,  5.00766047e-03,
        3.89627179e-12,  

In [54]:
# 年間コストを算出するため、最適値に365を乗じていっています。
annual_cost = 0
for i, fa in enumerate(food_amount):
    if fa.value > 0.001:
        print("{0} : ${1}".format(df_foods.food[i], fa.value))
        annual_cost += 365 * fa.value
        
print("上記がコスト最小となる最適な食材の組み合わせで、年間{0}ドルで済む".format(annual_cost))

Wheat Flour (Enriched) : $0.029519061688759233
Liver (Beef) : $0.0018925572778251906
Cabbage : $0.011214435241318356
Spinach : $0.00500766046924744
Navy Beans, Dried : $0.06102856345707608
上記がコスト最小となる最適な食材の組み合わせで、年間39.661731518992596ドルで済む


CVXPYで解くと全ての食材をごく少量買うという解が出たため、しきい値を 0.001ドル(0.1セント)としたところ、pulpの解に一致した。

＜pulpの解＞  
Wheat Flour (Enriched) : 0.029519062  
Liver (Beef) : 0.0018925573  
Cabbage : 0.011214435  
Spinach : 0.0050076605     
Navy Beans, Dried : 0.061028564  
上記がコスト最小となる最適な食材の組み合わせで、年間39.661731762ドルで済む  